In [1]:
import torch
from copy import deepcopy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer



C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset('ag_news')
train_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
validation_test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(1000))])
validation_ds = Dataset.from_dict(validation_test_ds[0:500])
test_ds = Dataset.from_dict(validation_test_ds[500:1000])

data = test_ds





Using custom data configuration default
Reusing dataset ag_news (C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 501.29it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-dd0ff9596fea92b0.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\ag_news\default\0.0.0\bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548\cache-12f3c4e4bf422cce.arrow


In [3]:
display(data)
display(data[0:5])

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

{'text': ['A Sneak Peek at Trillian 3.0 Instant Messaging The popular IM consolidation service adds audio and video chat.',
  'New Technology Powers Fuel Cells A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around \\$90, its Japanese inventors claim.',
  "Court rules against state Web-blocking law A Pennsylvania law requiring Internet service providers to block Web sites deemed by the state's prosecuting attorneys to be child pornography has been reversed by a U.S. federal court on free-speech grounds.",
  'Braves, Smoltz agree to new deal Atlanta, GA (Sports Network) - The Atlanta Braves announced Thursday that the team has come to terms with longtime pitcher John Smoltz to a new two-year contract with a club option for the 2007 season.',
  "US Supreme Court asked to rule on homosexuals' right to adopt children (AFP) AFP - The American Civil Liberties Union, the leading US civil rights group, 

In [4]:
import textattack
dataset = textattack.datasets.HuggingFaceDataset(data)
display(dataset)

In [5]:
#attack the original model using TextFooler
device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu") 
model_path = r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\models\Antonym_model\with_pos\allow_verb_noun_swap=True\run3'
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=4)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)



In [6]:
from textattack.attack_recipes.textfooler_jin_2019 import TextFoolerJin2019
attack = TextFoolerJin2019.build(model_wrapper)
attack.cuda_()
display(attack)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

In [7]:
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

attack_args = AttackArgs(num_examples=-1, random_seed = 765,
                         log_to_csv = r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\results\Antonym\with_pos\allow_verb_noun_swap=True\text_fooler_attack_results\run3\results.csv', 
                         csv_coloring_style = 'file')

attacker = Attacker(attack, dataset, attack_args)

attack_results = attacker.attack_dataset()


textattack: Logging to CSV at path C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\results\Antonym\with_pos\allow_verb_noun_swap=True\text_fooler_attack_results\run3\results.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|▏                                                                               | 1/500 [00:19<2:44:58, 19.84s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                          | 1/500 [00:20<2:48:07, 20.22s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[3 (98%)]] --> [[0 (62%)]]

[[A]] [[Sneak]] Peek at Trillian 3.0 Instant Messaging The [[popular]] IM [[consolidation]] service [[adds]] [[audio]] and [[video]] [[chat]].

[[another]] [[Undercover]] Peek at Trillian 3.0 Instant Messaging The [[contemporary]] IM [[reunification]] service [[insert]] [[vocal]] and [[footage]] [[talk]].




[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   0%|                          | 2/500 [00:27<1:52:52, 13.60s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   0%|                          | 2/500 [00:27<1:52:57, 13.61s/it]

...

--------------------------------------------- Result 498 ---------------------------------------------
[[1 (96%)]] --> [[0 (70%)]]

[[Vieira]] slams Spain #39;s stance on racism Patrick Vieira has fiercely criticised Spain #39;s record in combating racism in [[football]] and paid tribute to [[England]] #39;s record in acting against it.

[[Vera]] slams Spain #39;s stance on racism Patrick Vieira has fiercely criticised Spain #39;s record in combating racism in [[ballon]] and paid tribute to [[Anglican]] #39;s record in acting against it.




[Succeeded / Failed / Skipped / Total] 359 / 95 / 44 / 498: 100%|█████████████████▉| 499/500 [2:10:12<00:15, 15.66s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 359 / 96 / 44 / 499: 100%|█████████████████▉| 499/500 [2:10:12<00:15, 15.66s/it]

--------------------------------------------- Result 499 ---------------------------------------------
[[0 (98%)]] --> [[[FAILED]]]

Parliament passes no-confidence bid NAIROBI, Kenya -- Somalia's parliament passed a motion of no-confidence against the new prime minister and his Cabinet yesterday, an official said, effectively sacking a government that had been expected to restore order to the country after 13 years of anarchy and war. The deputy speaker of the 275-member transitional parliament, Dalhar Omar, said 153 members voted against Prime Minister ...




[Succeeded / Failed / Skipped / Total] 359 / 96 / 44 / 499: 100%|██████████████████| 500/500 [2:10:12<00:00, 15.63s/it]C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 360 / 96 / 44 / 500: 100%|██████████████████| 500/500 [2:10:12<00:00, 15.63s/it]

--------------------------------------------- Result 500 ---------------------------------------------
[[3 (51%)]] --> [[2 (59%)]]

Yahoo! Tunes In to Musicmatch Close watchers of the [[online]] music business no doubt noted yesterday #39;s announcement of Yahoo! #39;s (Nasdaq: YHOO) purchase of Musicmatch with interest.

Yahoo! Tunes In to Musicmatch Close watchers of the [[website]] music business no doubt noted yesterday #39;s announcement of Yahoo! #39;s (Nasdaq: YHOO) purchase of Musicmatch with interest.



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 360    |
| Number of failed attacks:     | 96     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 19.2%  |
| Attack success rate:          | 78.95% |
| Average perturbed word %:     | 24.18% |
| Average num. words per input: | 38.67  |
| Avg nu

In [8]:
import pandas as pd
pd.options.display.max_colwidth = 480 # increase colum width so we can actually read the examples

logger = CSVLogger(color_method='html')

for result in attack_results:
    logger.log_attack_result(result)

from IPython.core.display import display, HTML
display(HTML(logger.df[['original_text', 'perturbed_text']].to_html(escape=False)))
results_html = logger.df[['original_text', 'perturbed_text']].to_html(escape=False)
path = r'C:\Users\DELL\Text_Augmentation\ag_news_dataset_3runs\results\Antonym\with_pos\allow_verb_noun_swap=True\text_fooler_attack_results\run3\results.html'
text_file = open(path, "w")
text_file.write(results_html.encode("gbk", 'ignore').decode("gbk", "ignore"))
text_file.close()
logger.flush()

textattack: Logging to CSV at path results.csv
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated 

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

...

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\textattack\loggers\csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

,original_text,perturbed_text
0,A Sneak Peek at Trillian 3.0 Instant Messaging The popular IM consolidation service adds audio and video chat.,another Undercover Peek at Trillian 3.0 Instant Messaging The contemporary IM reunification service insert vocal and footage talk.
1,"New Technology Powers Fuel Cells A new fuel cell for notebook PCs, more compact and powerful than competing technologies, could be on the market in early 2006 at a price of around \$90, its Japanese inventors claim.","New Tech Powers Fuel Cells A new fuels cell for cell accoutrements, more compact and powerful than competing tech, could be on the market in early 2006 at a price of around \$90, its Japanese ingenuity claim."
2,Court rules against state Web-blocking law A Pennsylvania law requiring Internet service providers to block Web sites deemed by the state's prosecuting attorneys to be child pornography has been reversed by a U.S. federal court on free-speech grounds.,Court provision against state Web-blocking act A Pa law demanding Website services providers to block Web sites deemed by the state's prosecuting barrister to be child pornography had been reversed by a oder.S. govt judiciary on free-speech grounds.
3,"Braves, Smoltz agree to new deal Atlanta, GA (Sports Network) - The Atlanta Braves announced Thursday that the team has come to terms with longtime pitcher John Smoltz to a new two-year contract with a club option for the 2007 season.","Laroche, Smoltz subscribed to new bargains Nancy, AG (Sports Networks) - The Atlanta Dodger announced Wed that the devices owns entries to lexicon with protracted pitching June Smoltz to a ny two-year market with a cabaret alternatives for the 2007 season."
4,"US Supreme Court asked to rule on homosexuals' right to adopt children (AFP) AFP - The American Civil Liberties Union, the leading US civil rights group, petitioned the US Supreme Court to rule on the right of homosexuals to adopt children.","US Supreme Court asked to rule on homosexuals' right to adopt kiddies (AFP) AFP - The American Civil Liberties Union, the leading US civil rights bundled, petitioned the US Supreme Court to rule on the right of homosexuals to adopt babes."
5,"Eagles Lose OL Andrews PHILADELPHIA (Sports Network) - Offensive lineman Shawn Andrews, Philadelphia #39;s No. 1 draft pick this year, suffered a fractured right leg in Sunday #39;s game against the New York Giants.","Penguins Forfeit OL Andrew PHILLY (Sport Grates) - Detestable linemen Shawn Roberts, Chicago #39;s No. 1 draft selections this year, undergoing a fractured right kneecaps in Sundays #39;s games against the New York Barons."
6,"Pacers 89, Raptors 86 Jamaal Tinsley scored 17 of his 22 points in the second half to lead the Indiana Pacers to an 89-86 victory over the Toronto Raptors, who traded All Star Vince Carter to New Jersey earlier Friday.","Lebron 89, Raptors 86 Akeem Teague obtained 17 of his 22 points in the second halved to promote the Indiana Cavaliers to an 89-86 victors over the Canadiens Hawks, who redeemed Generals Superstar Vinny Johnson to Roman Jumper soon Thu."
7,Profile: Leftist Vazquez wins Uruguay presidential poll Uruguay #39;s leftist candidate Tabare Vazquez won a historic victory in Sunday #39;s presidential elections with more than 50 percent of the ballot.,Contours: Left Vazquez earn Ur senatorial suffrage Ur #39;s socialist competitor Tabare Vazquez won a historical wins in Fri #39;s senatorial valda with more than 50 percentages of the ballots.
8,Villeneuve and Sauber With Jacques Villeneuve testing for Renault today at Silverstone there has been the perhaps inevitable speculation that this could change things for next season when Giancarlo Fisichella is due to join Fernando Alonso at Renault F1.,Prost and Sauber Of Jacobo Piquet tested for Renaud sonntag at Silverstone there was been the admittedly undeniable assumed that this could revisions points for next season when Gianni Fisichella is due to competed Lewis Montoya at Rino F1.
9

In [9]:
import textattack

attack_log_manager = textattack.loggers.AttackLogManager()


for result in attack_results:
    attack_log_manager.log_result(result)

attack_log_manager.enable_stdout()
attack_log_manager.log_summary()

attack_log_manager.flush()


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 360    |
| Number of failed attacks:     | 96     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 91.2%  |
| Accuracy under attack:        | 19.2%  |
| Attack success rate:          | 78.95% |
| Average perturbed word %:     | 24.18% |
| Average num. words per input: | 38.67  |
| Avg num queries:              | 340.44 |
+-------------------------------+--------+